In [1]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
import pyspark.sql as sql
from pyspark.sql.functions import col

In [2]:
#sc.stop()

NameError: name 'sc' is not defined

In [ ]:
conf = SparkConf().setAppName("Lr1").setMaster('yarn')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [ ]:
tripData = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y H:m')\
.csv("trips.csv")

tripData.printSchema()

stationData = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y')\
.csv("stations.csv")

stationData.printSchema()

Задание 1. Найти велосипед с максимальным временем пробега.

In [ ]:
tripData.createOrReplaceTempView("Trips")

In [ ]:
spark.sql("SELECT bike_id, sum(duration) duration\
        FROM Trips\
        GROUP BY bike_id\
        ORDER BY duration DESC\
        LIMIT 1").show()

Задание 2. Найти наибольшее геодезическое расстояние между станциями.

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def geodesic_Haversine(lat1, lon1, lat2, lon2):
    dlon = radians(lon2) - radians(lon1)
    dlat = radians(lat2) - radians(lat1)
    
    # формула Хаверсина
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    return 6373 * 2 * atan2(sqrt(a), sqrt(1 - a))

In [ ]:
joined_stations = stationData.crossJoin(stationData)
joined_stations.printSchema()

In [ ]:
print(joined_stations.rdd.map(lambda x: geodesic_Haversine(x[2], x[3], x[9], x[10])).max())

Задание 3. Найти путь велосипеда с максимальным временем пробега через станции

In [ ]:
filtered_stations = joined_stations.rdd.filter(lambda l: (l[1] == temp[0].start_station_name) and (l[8] == temp[0].end_station_name))

print(filtered_stations.map(lambda x: geodesic_Haversine(x[2], x[3], x[9], x[10])).collect())

Задание 4 - Найти количество велосипедов в системе

In [ ]:
spark.sql("SELECT COUNT(DISTINCT bike_id) count\
        FROM Trips").show()

Задание 5. Найти пользователей потративших на поездки более 3 часов

In [ ]:
spark.sql("SELECT id, SUM(duration) sum\
        FROM Trips\
        GROUP BY id\
        HAVING sum > 10800").show()